<a href="https://colab.research.google.com/github/reyllama/TTIO/blob/master/01_CorrMatrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving stocks.csv to stocks.csv


In [6]:
import io
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

df = pd.read_csv(io.BytesIO(uploaded['stocks.csv']))
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,stock_id,name,date,opening_price,highest_price,lowest_price,closing_price,trading_volume
0,000020,동화약품,2016-01-04,8130,8150,7920,8140,281440
1,000020,동화약품,2016-01-05,8040,8250,8000,8190,243179
2,000020,동화약품,2016-01-06,8200,8590,8110,8550,609906
3,000020,동화약품,2016-01-07,8470,8690,8190,8380,704752
4,000020,동화약품,2016-01-08,8210,8900,8130,8770,802330


In [9]:
def year_parser(column):
  return int(column[0:4])

def month_parser(column):
  return int(column[5:7])

df['year'] = df['date'].apply(year_parser)
df['month'] = df['date'].apply(month_parser)

df.head()

,stock_id,name,date,opening_price,highest_price,lowest_price,closing_price,trading_volume,year,month
0,000020,동화약품,2016-01-04,8130,8150,7920,8140,281440,2016,1
1,000020,동화약품,2016-01-05,8040,8250,8000,8190,243179,2016,1
2,000020,동화약품,2016-01-06,8200,8590,8110,8550,609906,2016,1
3,000020,동화약품,2016-01-07,8470,8690,8190,8380,704752,2016,1
4,000020,동화약품,2016-01-08,8210,8900,8130,8770,802330,2016,1


In [92]:
df1 = df[(df['year']==2018)&(df['month'].isin([10,11,12]))]
df1.shape

(136358, 10)

In [93]:
df1.head(5)

,stock_id,name,date,opening_price,highest_price,lowest_price,closing_price,trading_volume,year,month
675,000020,동화약품,2018-10-01,11250,11500,11100,11150,42239,2018,10
676,000020,동화약품,2018-10-02,11250,11250,10850,10900,84470,2018,10
677,000020,동화약품,2018-10-04,10950,11100,10800,10950,41439,2018,10
678,000020,동화약품,2018-10-05,10800,10900,10350,10600,113530,2018,10
679,000020,동화약품,2018-10-08,10600,10650,10100,10100,164828,2018,10


In [56]:
# print(len(df1))
# df1.drop_duplicates(subset=['name', 'date']).shape

In [57]:
# reg = df1.drop_duplicates(subset=['name', 'date']).index
# df1.drop(reg).drop_duplicates(subset='name')

In [94]:
df2 = df1[['stock_id', 'date', 'closing_price']].pivot(index='date', columns='stock_id', values='closing_price').pct_change()

In [95]:
df2.head()

stock_id,30,1799,2000,2005,2250,3460,3465,3470,3475,3480,3490,3495,3520,3530,3535,3540,3545,3547,3550,3555,3560,3570,3580,3610,3620,3650,3670,3680,3690,3720,3780,3800,3830,3850,3920,3925,3960,4000,4020,4060,...,298050,298380,298690,299170,299900,299910,300720,302430,303030,306200,306620,307070,307160,307180,307280,307750,307870,309930,310200,310840,310870,311270,900070,900080,900100,900110,900120,900140,900250,900260,900270,900280,900290,900300,900310,900340,950110,950130,950140,950160
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-02,-0.011834,-0.003846,0.084507,0.070323,-0.001832,-0.003509,0.000000,-0.038667,-0.024896,-0.047619,-0.030249,-0.007463,-0.052891,-0.036538,-0.008677,-0.015936,-0.001050,-0.003398,-0.009550,0.004684,-0.022556,0.000000,0.029661,-0.011278,-0.011038,-0.001248,-0.046961,-0.047859,-0.009852,-0.018051,-0.018651,-0.028234,0.011583,-0.013158,0.007657,0.000000,-0.008811,0.003968,-0.003565,-0.012346,...,-0.031802,NaN,NaN,-0.004706,NaN,NaN,0.009836,NaN,-0.002392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.025367,-0.040377,0.000000,-0.012793,0.004190,-0.024465,-0.020833,-0.010169,-0.016598,-0.012500,0.001443,-0.039474,NaN,-0.008264,-0.043922,-0.007366,-0.056962
2018-10-04,0.017964,-0.032819,-0.010390,-0.014455,-0.003670,0.000000,-0.007273,-0.008322,-0.010638,-0.030303,-0.001835,-0.011278,-0.019481,-0.035928,0.000000,-0.024291,-0.013670,-0.009091,-0.002755,-0.008159,-0.020513,0.004545,-0.041152,-0.011407,-0.020089,0.001250,0.013043,0.015873,0.009950,-0.006127,-0.040936,-0.012162,-0.007634,0.017778,0.016717,-0.004608,-0.004444,-0.007905,-0.032200,-0.028750,...,-0.029197,NaN,NaN,0.002364,NaN,NaN,-0.009740,NaN,-0.002398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.004902,0.000000,-0.043478,-0.033951,-0.028078,-0.043115,-0.028213,-0.039007,-0.047945,-0.033755,-0.025316,-0.007205,-0.046967,NaN,-0.025000,0.006410,-0.027829,0.023490
2018-10-05,-0.011765,0.007984,-0.045932,-0.001333,0.001842,0.007042,0.000000,-0.034965,-0.010753,0.012500,0.018382,0.011407,-0.039735,0.010352,-0.024070,-0.004149,-0.005330,0.002294,-0.006906,-0.004700,0.005236,0.030166,-0.051502,-0.013462,-0.009112,0.012484,-0.074392,0.000000,0.014778,0.009864,-0.022866,-0.004788,-0.023077,-0.030568,0.020927,0.016204,0.000000,-0.025896,-0.005545,-0.011583,...,-0.003759,NaN,NaN,0.002358,NaN,NaN,-0.039344,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.013699,-0.032258,-0.006390,-0.020000,-0.058140,-0.019355,-0.014760,-0.043165,-0.021834,-0.028139,-0.029028,-0.030801,NaN,-0.007326,-0.035032,-0.058206,-0.038251
2018-10-08,0.005952,-0.021782,0.011004,0.000000,0.000000,-0.003497,-0.003663,-0.015942,0.000000,0.016975,-0.003610,-0.003759,-0.009655,-0.030738,0.002242,-0.020833,-0.005359,0.002288,-0.030598,-0.011806,-0.018229,-0.016105,0.049774,-0.011696,-0.013793,-0.007398,0.034003,-0.049479,0.000000,-0.009768,0.009360,-0.003436,0.015748,0.027027,0.001464,-0.015945,-0.004464,0.001022,-0.003717,0.002604,...,-0.015094,NaN,NaN,-0.002353,NaN,NaN,0.000000,NaN,0.002404,-0.061644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.024242,-0.025723,-0.029478,-0.006173,-0.032895,0.003745,-0.026316,-0.008929,0.002227,0.044843,-0.027542,NaN,0.025830,0.019802,-0.033435,-0.003409


In [96]:
df2 = df2.iloc[1:, :]
df2.head()
cor1 = df2.corr()
cor1.head()

stock_id,30,1799,2000,2005,2250,3460,3465,3470,3475,3480,3490,3495,3520,3530,3535,3540,3545,3547,3550,3555,3560,3570,3580,3610,3620,3650,3670,3680,3690,3720,3780,3800,3830,3850,3920,3925,3960,4000,4020,4060,...,298050,298380,298690,299170,299900,299910,300720,302430,303030,306200,306620,307070,307160,307180,307280,307750,307870,309930,310200,310840,310870,311270,900070,900080,900100,900110,900120,900140,900250,900260,900270,900280,900290,900300,900310,900340,950110,950130,950140,950160
stock_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
30,1.000000,-0.031108,0.123098,0.042424,0.069366,0.255768,0.045098,0.219867,0.423979,0.202327,0.348845,0.095451,0.127678,0.265388,0.239842,0.358036,0.387328,0.378986,0.323921,0.106572,0.228220,0.149695,0.121851,0.243280,0.034258,0.158633,0.255121,0.297750,0.479390,0.248757,0.071715,-0.056455,0.197633,0.101580,0.266127,0.075271,0.151981,0.248443,0.293451,0.229313,...,0.100874,-0.063177,NaN,0.282084,-0.019544,-0.024293,0.028371,-0.292516,0.099016,0.154462,0.113418,-0.010285,0.011055,-0.827898,0.062910,-0.097699,-0.230725,-0.007568,-0.508336,-0.710774,-0.136668,NaN,0.221744,0.288565,-0.061807,0.137228,0.147091,0.365446,0.141144,0.175922,0.077104,0.029827,0.113302,0.200479,0.247093,-0.202346,0.159265,0.115080,-0.010114,0.124986
1799,-0.031108,1.000000,-0.065331,-0.034314,-0.069900,-0.024279,0.288837,0.187517,0.085948,0.060187,0.149162,0.052628,0.016385,0.051202,-0.008140,-0.021478,0.062638,0.003809,0.164427,0.221507,0.139697,-0.109128,0.110801,0.113966,-0.074401,0.116694,0.012333,0.003990,0.025804,0.087472,-0.064687,0.144262,0.033459,-0.202492,-0.090105,-0.111191,0.125086,-0.025172,0.142864,-0.080570,...,0.006683,-0.391936,NaN,0.134598,0.053982,-0.086954,-0.127132,0.111701,0.083893,0.065323,0.019626,0.137066,0.185553,-0.829138,0.278074,0.257729,0.085028,-0.600648,-0.610684,-0.543324,-0.462519,NaN,0.120551,0.200586,0.090203,0.008986,0.075263,0.212503,0.123164,-0.061716,0.041761,0.045903,0.093120,-0.057727,0.069460,0.051865,0.079785,-0.034981,0.120265,-0.144792
2000,0.123098,-0.065331,1.000000,0.902140,0.103839,0.066818,-0.117785,0.104052,0.013438,-0.097010,0.012628,0.014460,-0.020870,0.040245,0.003195,0.081440,0.076129,0.024686,-0.020431,0.039543,-0.022954,0.071850,0.133692,-0.097448,0.069803,0.156643,0.132821,0.017133,-0.036721,-0.279802,-0.251028,-0.092296,0.162884,-0.035194,-0.026027,-0.068943,0.124430,0.113234,0.048862,0.037659,...,-0.033577,NaN,NaN,0.000702,NaN,-0.072399,0.183812,NaN,-0.180610,0.239396,-0.029694,0.028972,0.172193,NaN,-0.100423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005635,0.174062,0.032794,0.143241,0.114350,0.081048,0.006956,0.057827,0.141280,0.090458,0.171236,0.082111,0.126928,-0.024054,0.011942,-0.045459,-0.019126,0.149385
2005,0.042424,-0.034314,0.902140,1.000000,0.044865,0.067098,-0.084138,0.011937,-0.020275,-0.078539,0.004709,0.011104,-0.065509,-0.004937,-0.078954,0.016493,0.024494,0.015563,-0.099161,-0.003094,-0.098553,0.011632,0.041408,-0.228423,0.030169,0.162569,0.073461,0.002245,-0.095508,-0.310689,-0.320626,-0.058514,0.090724,-0.100163,-0.046235,-0.070477,0.062046,0.074805,0.007453,-0.006483,...,-0.076629,NaN,NaN,-0.067751,NaN,-0.020596,0.145411,NaN,-0.091098,0.194987,-0.099700,-0.083579,0.119994,NaN,-0.258209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.042217,0.111726,0.036181,0.134854,0.085756,0.045047,-0.053810,-0.003951,0.121535,0.059821,0.032996,0.004794,0.093132,-0.114815,-0.025313,-0.137075,-0.098130,0.069234
2250,0.069366,-0.069900,0.103839,0.044865,1.000000,0.186912,0.144954,0.137640,0.239803,0.156297,0.233400,0.440948,0.099268,0.144633,-0.059892,0.175974,0.116534,0.078503,0.196382,0.172921,0.070612,0.113800,0.027506,0.149893,0.391291,0.083179,-0.027433,0.099030,0.216473,-0.039578,0.079318,-0.031778,0.122663,0.190600,0.138033,0.126783,0.071748,0.076202,-0.024048,0.386623,...,0.217241,0.544261,NaN,0.051488,-0.160632,-0.274870,0.023689,-0.457640,0.170300,0.074694,-0.126783,-0.485176,-0.195965,0.246784,-0.001768,-0.537713,-0.545177,-0.364366,-0.11

In [97]:
cor1.to_csv('cor12.csv')